# Предварительная обработка
Нам понадобятся: 
* модуль `ijson` [отсюда](https://pypi.python.org/pypi/ijson) для парсинга данных в json-формате;
* стандартный модуль `time` для того, чтобы отобрать новости за нужные нам даты.

In [1]:
import ijson
import time

В `titles` будем заносить заголовки новостей, полученные из json-файла.

Также у нас есть 2 набора данных (и, соответственно 2 режима, которые определяются переменной `mode`): "full" - используется полный архив статей начиная с 1999 года по ноябрь 2017, "sample" - ограниченный набор статей для быстрых экспериментов.

Архив статей подготовлен [Ильдар Габдрахманов ildarchegg](https://habrahabr.ru/users/ildarchegg/) и описан в его [статье на Хабре](https://habrahabr.ru/post/343838/). [Прямая ссылка на файл](https://drive.google.com/open?id=1NlFuOjOt0oQ9Mx70Z7ZvfOsB3-1fCALp) (1.4 Гб в архиве, >7 Гб в распакованном виде)

Также выбираем период, за который мы хотим получить заголовки, и на которых будем тренировать сеть.

In [2]:
titles = []

# Все статьи
mode = "full"       

# Выборка (~800 статей)
# mode = "sample"

# Все статьи с 1999 по Ноябрь 2017
# start_date = "1999-01-01"

# Только статьи новой редакции Ленты
# start_date = "2014-04-01"

# Только статьи за 2017 год
start_date = "2017-01-01"

basetime = time.strptime(start_date, "%Y-%m-%d")

Парсим файл и выдергиваем те заголовки, которые попадают в нужный нам диапазон дат. Заметьте, что новости в файле не отсортированы по дате, поэтому придется пробежаться по всем записям.

In [3]:
objects = ijson.items(open('./data/lenta_'+mode+'.json', encoding='utf-8'), 'item')

items = (o for o in objects)

for item in items:
    if 'datetime' in item['page'][0] and time.strptime(item['page'][0]['datetime'][:10], "%Y-%m-%d") > basetime:
        titles.append(item['page'][0]['metaTitle'].lower())

### Сколько получилось заголовоков?

In [4]:
len(titles)

58517

### Пример заголовка

In [5]:
titles[0]

'рпцз призвала вынести ленина из мавзолея и начать декоммунизацию'

Сведем все заголовки в один длинный текст и посмотрим, сколько различных символов у нас встречается. Это важно, потому что чем больше символов, тем сложнее сети обучаться (в варианте с Char-RNN). 

In [24]:
text = " ".join(titles)
chars = sorted(list(set(text)))

print('Различных символов:', len(chars))

Различных символов: 111


In [25]:
print(chars)

[' ', '!', '"', '#', '$', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '«', '»', 'á', 'ä', 'ç', 'è', 'é', 'ï', 'ö', 'ø', 'ü', 'š', 'ɢ', '̶', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', 'є', 'і', '\u200d', '‑', '–', '—', '’', '…', '№']


Как видно выше, в тексте встречается много необычных (а потому, довольно редких) символов. Чтобы упростить задачу обучения, давайте часть их них удалим, а часть заменим на более часто встречающиеся:

In [26]:
chars_to_remove = ['\x00', '$', '%', '(', ')', '«', '»', '̆', '̶' ,'є', 'є', 'Є', '\n','\"','\'', '#', '&', '*', '+', '/', '<', '>', '@', '\\', '/', '_', '`', '\x7f', '\xa0', '«', '\xad', '¯', '»', '’', '…', '€', '№', 'ツ', '😂']
chars_to_replace = {
    '%': "процентов", 
    'š': "s", 
    '\u2009': " ", 
    '\u200a': " ", 
    '\u200d': "", 
    '\u200f': "", 
    '\u2028': "",
    'î': 'i', 
    'ø': 'o',
    'š': 's', 
    'á': 'a', 
    'ä': 'a', 
    'ç': 'c', 
    'è': 'e', 
    'é': 'e', 
    'ë': 'е', 
    'ё': 'e',
    'î': 'i', 
    'ï': 'i', 
    'ô': 'o', 
    'ö': 'o', 
    'ø': 'o', 
    'ü': 'u', 
    'š': 's', 
    'ɢ': 'g', 
    'і': 'i', 
    'ї': 'i', 
    'ӧ': 'o', 
    '‑': '-', 
    '–': '-', 
    '—': '-', 
    '―': '-',
    '─': '-'   
}

В `titles_clear` сохраним очищенные заголовки и добавляем точку в конце заголовка, чтобы было понятно, где конец заголовка:

In [27]:
titles_clear = []

for item in titles:

    for char in chars_to_remove:
        item = item.replace(char,'')

    for key, value in chars_to_replace.items():
        item = item.replace(key, value)
    
    titles_clear.append(item + '.')

#### Посмотрим, что получилось:

In [28]:
text = " ".join(titles_clear)
chars = sorted(list(set(text)))

print('Различных символов:', len(chars))
print('Средняя длина заголовка:', (len(text) / len(titles_clear)))

Различных символов: 75
Средняя длина заголовка: 59.28164465027257


#### Сохраняем результат в файл, добавляя пробел после каждого заголовка для красоты:

In [29]:
res_file = open('./data/headers_' + mode + '.txt', 'w', encoding='utf-8')
for item in titles_clear:
    res_file.write("%s " % item)

# Посмотрим на данные
## Начальные слова
Интересно посмотреть на статистику начальных слов заголовков. Найдем самые популярные слова:

In [30]:
first_words = {}

for item in titles_clear:
    word = item.split(' ')[0]
    if word in first_words:
        first_words[word] += 1
    else:
        first_words[word] = 1
        
list_first_words = [{"word": key, "count": value} for key, value in first_words.items()]
list_first_words = sorted(list_first_words, key=lambda k: k['count'], reverse=True) 

print("Уникальных слов: ", len(list_first_words))

Уникальных слов:  12881


В принципе, если выбросить стоп-слова, типа "в", "на" и т.п., можно в общих чертах понять, о чем чаще всего пишут:

In [31]:
list_first_words[:20]

[{'count': 6746, 'word': 'в'},
 {'count': 1004, 'word': 'на'},
 {'count': 822, 'word': 'путин'},
 {'count': 817, 'word': 'сми'},
 {'count': 481, 'word': 'глава'},
 {'count': 460, 'word': 'трамп'},
 {'count': 390, 'word': 'россия'},
 {'count': 325, 'word': 'названы'},
 {'count': 276, 'word': 'умер'},
 {'count': 270, 'word': 'власти'},
 {'count': 261, 'word': 'песков'},
 {'count': 260, 'word': 'суд'},
 {'count': 254, 'word': 'порошенко'},
 {'count': 254, 'word': 'бывший'},
 {'count': 247, 'word': 'сша'},
 {'count': 242, 'word': 'российские'},
 {'count': 239, 'word': 'российский'},
 {'count': 235, 'word': 'у'},
 {'count': 222, 'word': 'полиция'},
 {'count': 214, 'word': 'президент'}]

Посчитаем такую же статистику для полных заголовков:

In [32]:
word_counts = {}

for word in text.split(" "):
    word = word.replace('.', '')
    if word in word_counts:
        word_counts[word] += 1
    else:
        word_counts[word] = 1
        
list_words = [{"word": key, "count": value} for key, value in word_counts.items()]
list_words = sorted(list_words, key=lambda k: k['count'], reverse=True) 

print("Уникальных слов:", len(list_words))

Уникальных слов: 68649


Ожидаемо, стоп-слова заняли весь топ. И это нормально, т.к. со стоп-слов заголовки начинаются реже, чем со смысловых.

In [33]:
list_words[:20]

[{'count': 27762, 'word': 'в'},
 {'count': 11207, 'word': 'на'},
 {'count': 6497, 'word': 'с'},
 {'count': 6082, 'word': 'о'},
 {'count': 5433, 'word': 'и'},
 {'count': 3791, 'word': 'за'},
 {'count': 3175, 'word': 'по'},
 {'count': 3071, 'word': 'россии'},
 {'count': 2780, 'word': 'из'},
 {'count': 2189, 'word': 'сша'},
 {'count': 2074, 'word': 'для'},
 {'count': 1919, 'word': 'от'},
 {'count': 1570, 'word': 'из-за'},
 {'count': 1540, 'word': 'к'},
 {'count': 1249, 'word': 'об'},
 {'count': 1082, 'word': 'у'},
 {'count': 1058, 'word': 'после'},
 {'count': 967, 'word': 'рассказал'},
 {'count': 953, 'word': 'назвал'},
 {'count': 948, 'word': 'сми'}]

## Нормализация
В получившемся списке различные формы одного слова по понятным причинам считаются как разные слова. Чтобы исправить это и посмотреть, повлияет ли это на результаты обучения, нормализуем слова (также для этого процесса можно встретить термин "лемматизация"), используя библиотеку [pymorphy2](https://pymorphy2.readthedocs.io/en/latest/). 

Для начала отделим знаки пунтктуации от предыдущих слов пробелами, т.к. `pymorphy` считает слова "машина" и "машина." (с точкой) разными, и для второго случая она не сможет найти нормальную форму. Также удалим все двойные пробелы, если такие появятся.

In [34]:
chars_to_replace_for_norm = {
    '!': ' ! ',
    ',': ' , ',
    '-': ' - ',
    '. ': ' . ',
    '  ': ' '
}

for key, value in chars_to_replace_for_norm.items():
    text = text.replace(key, value)

Проходимся по тексту и приводим каждое слово к наиболее вероятной по мнению `pymorphy` нормальной форме (может занять довольно много времени, если текст длинный):

In [37]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
text_norm = []
text_split = text.split(' ')

for i in range(len(text_split)):
    word = text_split[i].replace('. ', '')
    norm = morph.parse(word)[0].normal_form
    text_norm.append(norm)

Посмотрим, что получилось:

In [38]:
print(text_norm[:100])

['рпцз', 'призвать', 'вынести', 'ленин', 'из', 'мавзолей', 'и', 'начать', 'декоммунизация', '.', 'найти', 'тело', 'пропасть', 'моряк', 'с', 'американский', 'эсминец', '.', 'полиция', 'мэриленд', 'арестовать', 'подозревать', 'в', 'убийство', 'россиянин', 'зиберов', '.', 'концерт', 'рэпер', 'баста', 'в', 'одесса', 'отменить', 'после', 'угроза', 'украинский', 'националист', '.', 'более', '200', 'тысяча', 'человек', 'выступить', 'против', 'презумпция', 'доверие', 'к', 'полицейский', '.', 'мутко', 'раскритиковать', 'газзаев', 'за', 'ответ', 'путин', 'в', 'время', 'прямая', 'линия', '.', 'у', 'я', 'не', 'железный', 'яйцо', '.', 'ковалев', 'прокомментировать', 'два', 'подряд', 'поражение', 'от', 'уорд', '.', 'в', 'фсвтс', 'прокомментировать', 'поставка', 'корабельный', 'вертолёт', 'ка', '-', '52к', 'в', 'египет', '.', 'украинский', 'националист', 'отобрать', 'флаг', 'лгбт', 'у', 'участник', 'гей', '-', 'парад', 'в', 'киев', '.', 'по']


Выглядит неплохо. Посчитаем обновленную статистику:

In [39]:
word_norm_counts = {}

for word in text_norm:
    if word in word_norm_counts:
        word_norm_counts[word] += 1
    else:
        word_norm_counts[word] = 1
        
list_words_norm = [{"word": key, "count": value} for key, value in word_norm_counts.items()]
list_words_norm = sorted(list_words_norm, key=lambda k: k['count'], reverse=True) 

print("Уникальных слов:", len(list_words_norm))

Уникальных слов: 29871


Уникальных слов теперь более чем в 2 раза меньше! А топ слов выглядит так:

In [40]:
list_words_norm[:20]

[{'count': 58796, 'word': '.'},
 {'count': 28671, 'word': 'в'},
 {'count': 11271, 'word': 'на'},
 {'count': 7772, 'word': '-'},
 {'count': 7337, 'word': 'о'},
 {'count': 7026, 'word': 'с'},
 {'count': 5437, 'word': 'и'},
 {'count': 5361, 'word': 'за'},
 {'count': 4381, 'word': 'из'},
 {'count': 4308, 'word': 'россия'},
 {'count': 3242, 'word': 'по'},
 {'count': 2520, 'word': 'назвать'},
 {'count': 2222, 'word': 'российский'},
 {'count': 2194, 'word': 'сша'},
 {'count': 2074, 'word': 'для'},
 {'count': 1919, 'word': 'от'},
 {'count': 1753, 'word': 'рассказать'},
 {'count': 1682, 'word': 'трамп'},
 {'count': 1569, 'word': 'к'},
 {'count': 1542, 'word': 'путин'}]

Собираем текст обратно, возвращаем знаки пунктуации на место и удаляем букву "ё", которую повставлял `pymorphy`:

In [41]:
text_norm_joined = " ".join(text_norm)
chars_to_replace = {
    ' - ': '-', 
    ' !': '!',
    ' ,': ',',
    ' .': '.',
    'ё' : 'е'
}
for key, value in chars_to_replace.items():
    text_norm_joined = text_norm_joined.replace(key, value)

Сохраняем вариант текста с нормализованными словами:

In [42]:
res_file = open('./data/headers_' + mode + '_norm.txt', 'w', encoding='utf-8')
res_file.write(text_norm_joined)

3380267